<a href="https://colab.research.google.com/github/nikshrimali/TSAI_END/blob/main/Capstone/PythonCodeGen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install torchtext==0.8.1

     |████████████████████████████████| 7.0MB 5.8MB/s 
     |████████████████████████████████| 776.8MB 24kB/s 
ERROR: torchvision 0.9.0+cu101 has requirement torch==1.8.0, but you'll have torch 1.7.1 which is incompatible.
  Found existing installation: torch 1.8.0+cu101
    Uninstalling torch-1.8.0+cu101:
      Successfully uninstalled torch-1.8.0+cu101
  Found existing installation: torchtext 0.9.0
    Uninstalling torchtext-0.9.0:
      Successfully uninstalled torchtext-0.9.0


In [1]:
# Importing torch and essential libraries
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchtext.data import Field, BucketIterator, TabularDataset


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Currently running on {device}')

import spacy
spacy_en = spacy.load('en')

import numpy as np
import pandas as pd
import os
import random
import math
import time
import json
import random

import logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s : %(levelname)s : %(message)s')
logger = logging.getLogger('PythonCodeGen')



Currently running on cuda


In [3]:
# Spacy model for 
!python -m spacy download en

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


Now to set the results in a processable format, we will set the random seed values, Please note that the random seed values only gaurantee results in same runtime, results may vary after runtime is refreshed, to know more about refreshing runtimes, you can refer this artcicle

In [4]:
SEED = 1234
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

The below function helps us to tokenize the text we input

In [5]:
def tokenize_en(text):
  return [tok.text for tok in spacy_en.tokenizer(text)]

In [ ]:
# Divide the data into statement and code format

datasets = [[]]
file_name = '/content/drive/MyDrive/english_python_data_cleaned.txt'

with open(file_name) as f:
  #my_dict = {"description":[],"code":[]}
  for line in f:
    if line.startswith('#'):
      comment = line.split('\n#')
      if datasets[-1] != []:
        # we are in a new block
        datasets.append(comment)
    else:
      stripped_line = line#.strip()
      if stripped_line:
        datasets[-1].append(stripped_line)

In [ ]:
STAT = Field(tokenize= tokenize_en, 
            init_token='<sos>', 
            eos_token='<eos>', 
            lower=True,
            batch_first = True)

CODE = Field(tokenize = tokenize_en, 
            init_token='<sos>', 
            eos_token='<eos>', 
            lower=True,
            batch_first = True)

In [ ]:
# Remove all the comments and the multi-line strings
def apply_transforms(text):
    '''Applies regex transforms on the data by removing comments and normalizing
    new lines and whitespaces'''
    text = str(text)
    text = re.sub(r'#.*?\\n','', text) # Removed all the comments
    text = text.replace(' ', ' ws ') # Replace whitespace with ' ws ' 
    text = text.replace("\\n", " nl ") # Replace \n with ' nl '
    return text


In [ ]:
STAT.build_vocab(train_data, min_freq=2)
CODE.build_vocab(, min_freq=2)

After building a vocab, next step is creating a bucket iterator. A bucket iterator helps us to batch together sentence of similar lengths. 

In [ ]:
BATCH_SIZE = 128

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size = BATCH_SIZE,
    device = device
)

In [ ]:
class Encoder(nn.Module):
    def __init__(self,
                 input_dim,
                 hid_dim,
                 n_Layers,
                 n_heads,
                 pf_dim,
                 dropout,
                 device,
                 max_length = 100):

        
        super().__init__()

        self.device = device

        # below we are breaking down the embedding into input and positional embedding
        self.tok_embd = nn.Embedding(num_embeddings=input_dim, embedding_dim=hid_dim)
        self.pos_embd = nn.Embedding(max_length, hid_dim)

        # We also add layers for multi-headed processing

        self.layers = nn.ModuleList([EncoderLayer(hid_dim,
                                                  pf_dim,
                                                  n_heads,
                                                  dropout,
                                                  device)
                                    for _ in range(n_Layers)])

        # When we add two embeddings, we multiply our embeddings with a scale parameter, which helps us to maintain
        # our values in a certain range

        self.scale = torch.sqrt(torch.FloatTensor([hid_dim])).to(device)

        # Creating linear layer to reduce the dimension of the vector


        # We also add a dropout value for regularization

        self.dropout = nn.Dropout(dropout)

    
    def forward(self, input_src, src_mask):

        # input_src = [batch_size, src_len]
        batch_size = input_src.shape[0]
        src_len = input_src.shape[1]

        # Is src_len same in all the cases?

        pos = torch.arange(0, src_len).unsqueeze(0).repeat(batch_size,1).to(self.device)
        # print(f'Pos shape {pos.shape}, {input_src.shape}')
        #pos = [batch_size, src_len]

        input_embd = self.tok_embd(input_src)
        pos_embd = self.pos_embd(pos)

        # input_embd = pos_embd = [batch_size, src_len, embedding_dim]
        src = self.dropout(input_embd*self.scale + pos_embd)
        # src = [batch_size, src_len, hid_dim]

        # what does encoder returns?
        # 
        for layer in self.layers:
            src = layer(src, src_mask)

        return src


Encoder Layer -  This layer creates the multiple encoder layer which consists of Pointwise Feed forward and multi-headed attention networks, the code is mentioned below. This layer basically calls attention layer  and pointwise layers and also applies a norm to their outputs

In [ ]:
class EncoderLayer(nn.Module):
    def __init__(self,
                 hid_dim,
                 pf_dim,
                 n_heads,
                 dropout,
                 device):
        
        super().__init__()
        self.self_attn_layer_norm = nn.LayerNorm(hid_dim) # Layer norm after attention layer
        self.self_ff_layer_norm = nn.LayerNorm(hid_dim) # Layer norm after feed forward layer
        self.self_attention = MultiHeadAttention(hid_dim, n_heads, dropout, device) # Multi-head attention layer
        self.positionwise_feedforward = PositionwiseFeedforwardLayer(hid_dim, pf_dim, dropout)
        self.dropout = nn.Dropout(dropout) 
    
    def forward(self, src, src_mask):
        # what is src mask?????
        # src = [batch_size, src_len, hid_dim]
        # Why source has the hidden dim?
        # src_mask = [batch_size, 1, 1, src_len]
        
        _src, _ = self.self_attention(src, src, src, src_mask) # Self attention layer
        src = self.self_attn_layer_norm(self.dropout(_src) + src) # Add and Norm layer with residual connection

        # src = [batch_size, src_len, hid_dim]
        # Pointwise feedforward
        _src = self.positionwise_feedforward(src)
        src = self.self_ff_layer_norm(self.dropout(_src) + src)
        # src = [batch_size, src len, hid_dim]
        return src

![](https://raw.githubusercontent.com/bentrevett/pytorch-seq2seq/9479fcb532214ad26fd4bda9fcf081a05e1aaf4e/assets/transformer-attention.png)

## Scaled Dot-product attention - 

In scaled dot product attention query and key via matrix multiplication, after this the output is scaled, masked and softmax is applied to it. This attenion is then multiplied with the value to get contextual information.

## Multi-Head Attention
Why we call it as multi-head is it takes data into 3 forms Key, Query and the Value. Doing a matrix multiplication of Key with Query gives us attention values, these attention values are then multiplied by the Value which gives us contextualized embeddings.

Each embedding is divided into certain heads (which is also knows as n-heads), the total hid dim are divided into n-heads, these n-heads then act as the channels which contains relevant info.


In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self,
                 hid_dim,
                 n_heads,
                 dropout,
                 device):
        
        super().__init__()
        self.hid_dim = hid_dim
        self.n_heads = n_heads
        self.head_dim = hid_dim//n_heads

        self.fc_q = nn.Linear(hid_dim, hid_dim)
        self.fc_k = nn.Linear(hid_dim, hid_dim)
        self.fc_v = nn.Linear(hid_dim, hid_dim)

        self.fc_o = nn.Linear(hid_dim, hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        self.scale = torch.sqrt(torch.FloatTensor([self.head_dim])).to(device)
    
    def forward(self, query, key, value, mask=None):

        batch_size = query.shape[0]
        # query = [batch size, query_len, hid_dim]
        # key = [batch size, key_len, hid_dim]
        # value = [batch size, value_len, hid_dim]

        Q = self.fc_q(query)
        K = self.fc_k(key)
        V = self.fc_v(value)

        # Q = [batch size, query_len, hid_dim]
        # K = [batch size, key_len, hid_dim]
        # V = [batch size, value_len, hid_dim]

        # How hid_dim is divided into n_heads, head_dim?
        # print('Attention batch, heads, head_dim', batch_size, -1, self.n_heads, self.head_dim)

        Q = Q.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        K = K.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        V = V.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)

        # Q=K=V = [batch_size, n_heads, query/key/value_len, head_dim]

        # Transposing the input embedding and then doing matrix multiplication

        # [batch_size, src_len, src_len]
        energy = torch.matmul(Q, K.permute(0, 1, 3, 2))/ self.scale
        # print('Shape of energy', energy.shape)
        # energy = [batch_size, n_heads, query_len, key_length]
        # As you can notice the head dim is not there in the energy vector. only query_len and key_len are there

        # This energy is then multiplied with the value to calculate attention

        if mask is not None:
            # If mask values are close to zero, set it to very small values, we do this because?
            energy = energy.masked_fill(mask == 0, -1e10)

        # After matrix mul of query and key, and scaling we will apply softmax to get the output in a distribution of 0 to +1.
        # This value will act as an attention vector for us

        attention = torch.softmax(energy, dim=-1)

        # Attention is then further multiplied by the values to get the contextual embeddings

        x = torch.matmul(self.dropout(attention), V)

        #x = [batch_size, query_length, n_heads, hid_dim]

        x = x.permute(0, 2, 1, 3).contiguous()
        #x = [batch_size, n_heads, query_length, hid_dim]

        x = x.view(batch_size, -1, self.hid_dim)

        #x = [batch_size, query_len, hid_dim]

        x = self.fc_o(x)

        #x = [batch_size, query_len, hid_dim]

        return x, attention

In [ ]:
class PositionwiseFeedforwardLayer(nn.Module):
    def __init__(self, hid_dim, pf_dim, dropout):
        super().__init__()

        self.fc1 = nn.Linear(hid_dim, pf_dim)
        self.fc2 = nn.Linear(pf_dim, hid_dim)

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # x = [batch_size, seq_len, hid_dim] 
        x = self.dropout(torch.relu(self.fc1(x)))
        x = self.fc2(x)
        # x = [batch_size, seq_len, hid_dim]

        return x

## Decoder
![](https://raw.githubusercontent.com/bentrevett/pytorch-seq2seq/9479fcb532214ad26fd4bda9fcf081a05e1aaf4e/assets/transformer-decoder.png)

Now comes the decoder part, the decoder takes encoder outputs and the target sentece to generate train the network and generate the outputs.

The decoder components are mostly similar to the encoder, although there are a few necessary changes that are done.

Starting from below we can see the same embeddings (target and the positional embeddings are applied.

These embeddings are then sent to masekd multi-head attention blocks, these blocks and theoutputs are then added with the residual outputs and then noramalized using batch norm (known as layer norm here).

Now these outputs are again sent the Multi-headed attention block along with the encoder outputs, this block generates a relationship between Source coming from encoder and the Target values.

Then these values are again added with the residual outputs and sent to linear layer. Finally softmax is applied to the results

In [ ]:
class Decoder(nn.Module):
    def __init__(self,
                 output_dim,
                 hid_dim,
                 n_layers,
                 n_heads,
                 pf_dim,
                 dropout,
                 device,
                 max_length=100):

        super().__init__()
        self.device = device
        self.dropout = nn.Dropout(dropout)

        self.tok_embd = nn.Embedding(output_dim, hid_dim)
        self.pos_embd = nn.Embedding(max_length, hid_dim)

        self.layers = nn.ModuleList([DecoderLayer(hid_dim, 
                                                  n_heads, 
                                                  pf_dim, 
                                                  dropout, 
                                                  device)
                                     for _ in range(n_layers)])
        
        self.scale = torch.sqrt(torch.FloatTensor([hid_dim])).to(device)
        self.fc_out = nn.Linear(hid_dim, output_dim)
    
    def forward(self, trg, enc_src, trg_mask, src_mask):

        # src = [batch_size, trg_len]

        batch_size = trg.shape[0]
        trg_len = trg.shape[1]

        pos = torch.arange(0, trg_len).unsqueeze(0).repeat(batch_size,1).to(device)

        # pos = [batch_size, trg_len]

        trg = self.dropout(self.tok_embd(trg)* self.scale + self.pos_embd(pos))

        for layer in self.layers:
            # Why src_mask and trg_mask
            trg, attention = layer(trg, enc_src, trg_mask, src_mask)
        
        output = self.fc_out(trg)
        # output = [batch_size, trg_len, output_dim, ]
        return output, attention

In [ ]:
class DecoderLayer(nn.Module):
    def __init__(self,
                 hid_dim,
                 n_heads,
                 pf_dim,
                 dropout,
                 device):
        super().__init__()

        self.self_attn_lyr_norm = nn.LayerNorm(hid_dim)
        self.enc_attn_lyr_norm = nn.LayerNorm(hid_dim)
        self.ff_layer_norm = nn.LayerNorm(hid_dim)
        self.self_attention = MultiHeadAttention(hid_dim, n_heads, dropout, device)
        self.encoder_attention = MultiHeadAttention(hid_dim, n_heads, dropout, device)
        self.positionwise_feedforward = PositionwiseFeedforwardLayer(hid_dim, pf_dim, dropout)

        self.dropout = nn.Dropout(dropout)

    def forward(self, trg, enc_src, trg_mask, src_mask):
        # trg = [batch_size, trg_len, hid_dim]
        # enc_src = [batch_size, src_len, hid_dim]
        # trg_mask = [batch_size, 1, trg_len, trg_len]
        # src_mask = [batch_size, 1, 1, src_len]

        # Self 
        # print('Target shape and mask', trg.shape, trg_mask.shape)
        _trg, _ = self.self_attention(trg, trg, trg, trg_mask)

        # Layer Norm - Dropout, Relu, residual connection
        trg = self.self_attn_lyr_norm(self.dropout(_trg) + trg)

        # query, key, value
        _trg, attention = self.encoder_attention(trg, enc_src, enc_src, src_mask)
        trg = self.enc_attn_lyr_norm(self.dropout(_trg) + trg)

        trg = self.positionwise_feedforward(trg)
        # trg = [batch_size, trg_len, hid_dim]
        # attention = [batch_size, n_heads, trg_len, src_len]

        trg = self.ff_layer_norm(trg + self.dropout(_trg))
        return trg, attention

## Seq2Seq
Now its time to build a connection for our data in model, it will travel from encoder to decoder. This class will take the below params

- Encoder Object
- Encoder Mask
- Decoder Object
- Decoder Mask
- Device
- Dropout Value


First we will create mask for our data, 

### But why we need mask??
We need mask because ......

After this we will cal

'SRC' which is also our source sentence is sent to encoder along with the 

'TRG' would be sent to our decoder, along with encoder outputs which is also our generated attention

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self,
                 encoder,
                 decoder,
                 src_pad_idx,
                 trg_pad_idx,
                 device):
        
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.src_pad_idx = src_pad_idx
        self.trg_pad_idx = trg_pad_idx
        self.device = device
        
    def make_src_mask(self, src):

        # src = [batch_size, src_len]
        src_mask = (src != self.src_pad_idx).unsqueeze(1).unsqueeze(2)

        # src_mask = [batch_size, 1, 1, src_len]

        return src_mask

    def make_trg_mask(self, trg):
        # trg = [batch_size, trg_len]
        trg_pad_mask = (trg != self.trg_pad_idx).unsqueeze(1).unsqueeze(2)
        # print(f'Make target mask {trg.shape}, {trg_pad_mask.shape}')
        # trg_mask = [batch_size, 1, 1, trg_len]
    
        trg_len = trg.shape[1]
        
        trg_sub_mask = torch.tril(torch.ones((trg_len, trg_len), device=self.device)).bool()
        
        #trg_mask = [batch_size, 1, trg_len, trg_len]
        
        trg_mask = trg_pad_mask & trg_sub_mask # What is this & operator???

        # trg_mask = [batch_size, 1, trg_len, trg_len]
        # print(f'Target mask shape make_trg_mask {trg_mask.shape}')

        return trg_mask

    def forward(self, src, trg):
        # src = [batch_size, src_len]
        # trg = [batch_size, trg_len]
        src_mask = self.make_src_mask(src)
        trg_mask = self.make_trg_mask(trg)

        enc_src = self.encoder(src, src_mask)

        # enc_src = [batch_size, src_len, hid_dim, output_dim]
        
        output, attention = self.decoder(trg, enc_src, trg_mask, src_mask)

        # output = [batch_size, trg_len, output_dim]
        # attention = [batch_size, n_heads, trg_len, src_len]

        return output, attention

In [ ]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
HID_DIM = 256
ENC_LAYERS = 3
DEC_LAYERS = 3
ENC_HEADS = 8
DEC_HEADS = 8
ENC_PF_DIM = 512
DEC_PF_DIM = 512
ENC_DROPOUT = 0.1
DEC_DROPOUT = 0.1

enc = Encoder(INPUT_DIM, 
              HID_DIM, 
              ENC_LAYERS, 
              ENC_HEADS, 
              ENC_PF_DIM, 
              ENC_DROPOUT, 
              device)


dec = Decoder(OUTPUT_DIM, 
              HID_DIM, 
              DEC_LAYERS, 
              DEC_HEADS, 
              DEC_PF_DIM, 
              DEC_DROPOUT,
              device)

In [ ]:
# This is a default dict which contains different stopwords, here we check index of <pad> token
SRC.vocab.stoi['<pad>']

1

In [ ]:
SRC_PAD_IDX = SRC.vocab.stoi[SRC.pad_token]
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]

model = Seq2Seq(enc, dec, SRC_PAD_IDX, TRG_PAD_IDX, device).to(device)

In [ ]:
print(SRC.vocab.stoi[SRC.pad_token])

1


In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 9,552,321 trainable parameters


In [ ]:
def initialize_weights(m):
    if hasattr(m, 'weight') and m.weight.dim() > 1:
        nn.init.xavier_uniform_(m.weight.data)

In [ ]:
model.apply(initialize_weights);

In [ ]:
LEARNING_RATE = 0.0005

optimizer = torch.optim.Adam(model.parameters(), lr = LEARNING_RATE)

In [ ]:
criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

In [ ]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.src
        trg = batch.trg

        # print(src.shape, trg.shape)
        
        optimizer.zero_grad()

        # print(trg[:,:-1].shape)
        
        output, _ = model(src, trg[:,:-1])
                
        #output = [batch size, trg len - 1, output dim]
        #trg = [batch size, trg len]
            
        output_dim = output.shape[-1]
            
        output = output.contiguous().view(-1, output_dim)
        trg = trg[:,1:].contiguous().view(-1)
                
        #output = [batch size * trg len - 1, output dim]
        #trg = [batch size * trg len - 1]
            
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.src
            trg = batch.trg

            output, _ = model(src, trg[:,:-1])
            
            #output = [batch size, trg len - 1, output dim]
            #trg = [batch size, trg len]
            
            output_dim = output.shape[-1]
            
            output = output.contiguous().view(-1, output_dim)
            trg = trg[:,1:].contiguous().view(-1)
            
            #output = [batch size * trg len - 1, output dim]
            #trg = [batch size * trg len - 1]
            
            loss = criterion(output, trg)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut6-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 0m 21s
	Train Loss: 4.562 | Train PPL:  95.762
	 Val. Loss: 3.148 |  Val. PPL:  23.294


KeyboardInterrupt: ignored

In [ ]:
model.load_state_dict(torch.load('tut6-model.pt'))

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

In [ ]:
def translate_sentence(sentence, src_field, trg_field, model, device, max_len = 50):
    
    model.eval()
        
    if isinstance(sentence, str):
        nlp = spacy.load('de')
        tokens = [token.text.lower() for token in nlp(sentence)]
    else:
        tokens = [token.lower() for token in sentence]

    tokens = [src_field.init_token] + tokens + [src_field.eos_token]
        
    src_indexes = [src_field.vocab.stoi[token] for token in tokens]

    src_tensor = torch.LongTensor(src_indexes).unsqueeze(0).to(device)
    
    src_mask = model.make_src_mask(src_tensor)
    
    with torch.no_grad():
        enc_src = model.encoder(src_tensor, src_mask)

    trg_indexes = [trg_field.vocab.stoi[trg_field.init_token]]

    for i in range(max_len):

        trg_tensor = torch.LongTensor(trg_indexes).unsqueeze(0).to(device)

        trg_mask = model.make_trg_mask(trg_tensor)
        
        with torch.no_grad():
            output, attention = model.decoder(trg_tensor, enc_src, trg_mask, src_mask)
        
        pred_token = output.argmax(2)[:,-1].item()
        
        trg_indexes.append(pred_token)

        if pred_token == trg_field.vocab.stoi[trg_field.eos_token]:
            break
    
    trg_tokens = [trg_field.vocab.itos[i] for i in trg_indexes]
    
    return trg_tokens[1:], attention

In [ ]:
def display_attention(sentence, translation, attention, n_heads = 8, n_rows = 4, n_cols = 2):
    
    assert n_rows * n_cols == n_heads
    
    fig = plt.figure(figsize=(15,25))
    
    for i in range(n_heads):
        
        ax = fig.add_subplot(n_rows, n_cols, i+1)
        
        _attention = attention.squeeze(0)[i].cpu().detach().numpy()

        cax = ax.matshow(_attention, cmap='bone')

        ax.tick_params(labelsize=12)
        ax.set_xticklabels(['']+['<sos>']+[t.lower() for t in sentence]+['<eos>'], 
                           rotation=45)
        ax.set_yticklabels(['']+translation)

        ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
        ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.show()

In [ ]:
example_idx = 8

src = vars(train_data.examples[example_idx])['src']
trg = vars(train_data.examples[example_idx])['trg']

print(f'src = {src}')
print(f'trg = {trg}')

In [ ]:
translation, attention = translate_sentence(src, SRC, TRG, model, device)

print(f'predicted trg = {translation}')

In [ ]:
example_idx = 6

src = vars(valid_data.examples[example_idx])['src']
trg = vars(valid_data.examples[example_idx])['trg']

print(f'src = {src}')
print(f'trg = {trg}')

In [ ]:
translation, attention = translate_sentence(src, SRC, TRG, model, device)

print(f'predicted trg = {translation}')

In [ ]:
example_idx = 10

src = vars(test_data.examples[example_idx])['src']
trg = vars(test_data.examples[example_idx])['trg']

print(f'src = {src}')
print(f'trg = {trg}')

In [ ]:
translation, attention = translate_sentence(src, SRC, TRG, model, device)

print(f'predicted trg = {translation}')

In [ ]:
from torchtext.data.metrics import bleu_score

def calculate_bleu(data, src_field, trg_field, model, device, max_len = 50):
    
    trgs = []
    pred_trgs = []
    
    for datum in data:
        
        src = vars(datum)['src']
        trg = vars(datum)['trg']
        
        pred_trg, _ = translate_sentence(src, src_field, trg_field, model, device, max_len)
        
        #cut off <eos> token
        pred_trg = pred_trg[:-1]
        
        pred_trgs.append(pred_trg)
        trgs.append([trg])
        
    return bleu_score(pred_trgs, trgs)

In [ ]:
bleu_score = calculate_bleu(test_data, SRC, TRG, model, device)

print(f'BLEU score = {bleu_score*100:.2f}')